# Exercise 04 - Due Friday, October 21 at 12pm

*Objectives*: Gain experience loading a CSV dataset into a database model you define yourself and using SQL to explore its contents. Explore the data by writing and executing a number of SQL queries using common syntax and functions and describing your findings.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with SQL queries in the space provided, unless a text answer is requested. Results should be correct, according to the embedded tests. The notebook itself should be completely reproducible; from start to finish, another person should be able to use the same code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking at each major step in each problem.

*Deadline*: Friday, October 21, 12pm. Submit your notebook to Blackboard and push it to your GitHub repository.

## Setup - obtain data and create database

The US City Open Data Census has data on [restaurant inspections from all over the country](http://us-city.census.okfn.org/dataset/food-safety).  Let's take a look at data from Albuquerque.  *Note*: metadata and other details for this dataset are available at http://data.cabq.gov/business/foodinspections/.

Fyi, to save you a step, I've converted this file into a clean CSV file by using the `dos2unix` command to change its line endings and to switch it to use commas instead of tabs using `csvformat -t inputfile.csv > outputfile.csv`.

First we download the dataset:

In [ ]:
!wget https://raw.githubusercontent.com/gwsb-istm-6212-fall-2016/syllabus-and-schedule/master/exercises/abq-food-inspections-fy2010.csv

## Problem 1 - examine the data (35 points)

Use `csvstat` and other tools as you see fit and observe its contents.  Review the metadata definitions linked to above.  Describe what you see:  What do all the columns mean?  Are there null values?  Which columns interest you the most?  Which columns present some complications or questions you would like to ask?

In [ ]:
!csvcut -n abq-food-inspections-fy2010.csv

In [ ]:
!shuf -n 10000 abq-food-inspections-fy2010.csv | csvstat

Insert a combination of code and text/markdown cells here to review the data and describe what you find.

column descriptions : 
    1. FACILITY_NAME : Name of the facility
    2. FACILITY_KEY : The unique key of the facility
    3. SITE_ADDRESS : Street address of the facility
    4. CITY : City of the facility
    5. STATE : State of the facility                                                  
    6. ZIP : Zip of the facility                                                        
    7. OWNER_KEY : The unique key of the Owner of the facility                              
    8. OWNER_NAME : Name of the Owner of the facility                                       
    9. NATURE_OF_BUSINESS : The type of business                                           
    10. STREET_NUMBER : Street number of the facility                                      
    11. STREET_NAME : Street name of the facility                                          
    12. STREET_TYPE : Street type of the facility                               
    13. POST_DIRECTIONAL : Postal direction of the facility        
    14: PHONE : Phone number of the facility                                               
    15: PROGRAM_CATEGORY : Id that indicates the type of facility permit                   
    16: PROGRAM_CATEGORY_DESCRIPTION : Descript of the id that shows the type of facility permit
    17: INSPECTION_DATE : Date the inspection occurred                                     
    18: INSPECTION_TYPE : Code for type of inspection                              
    19: INSPECTION_DESC : Description of the inspection               
    20: SERIAL_NUM: Unique ID for the inspection daily activity           
    21: ACTION_CODE : Code of the action taken                            
    22: ACTION_DESC : Description of the action taken                
    23: RESULT_CODE : Code of the result of the inspection           
    24: RESULT_DESC : Description of the result of the inspection       
    25: VIOLATION_CODE : The code for the violation                     
    26: VIOLATION_DESC : Description of the violation that was found        
    27: INSPECTION_MEMO : Notes from the inspection made by the inspector

### Null value exsits in the 3,9,10,11,12,13,14,16,18,25,26,27 columns

### Which columns interest you the most?
What I am most interestred in is the violation description column which tells us the different types of violation that restaurants commit. 

### Which columns present some complications or questions you would like to ask?
While being interested in the results of different kinds of voilations, I want to know that which type of the business (NATURE_OF_BUSINESS) is the one most easily commit violations. And what are the most frequent violations (VIOLATION_CODE, VIOLATION_DESC) they commit.

## Problem 2 - define a database model and load the data (35 points)

Based on what you found above, create and connect to a new database, define a database table in it, and load this dataset into it.  You may use either of the methods for this step you have seen in earlier class notebooks.  You may choose to eliminate variables/columns if they are not relevant or interesting to you - explain your reasoning if you do.  Either way, you should load a majority of the columns present in the source dataset into the database and all of its rows.

Once your data has loaded successfully, run a `COUNT(*)` query to verify that all the data has loaded correctly.

Insert a combination of code and text/markdown cells here to connect to your database, define your table, load its data, and verify the loaded records.

In [ ]:
%load_ext sql

In [ ]:
!echo 'redspot' | sudo -S service postgresql restart

In [ ]:
!createdb -U dbuser ex4

In [ ]:
%sql postgresql://dbuser@localhost:5432/ex4

In [ ]:
%%sql
DROP TABLE IF EXISTS insp;
CREATE TABLE insp(
    FACILITY_NAME CHAR(202),
    FACILITY_KEY CHAR(20),
    SITE_ADDRESS CHAR(102),
    CITY CHAR(52),
    STATE CHAR(6),                                                        
    ZIP CHAR(22),
    OWNER_KEY CHAR(26),                                    
    OWNER_NAME CHAR(202), 
    NATURE_OF_BUSINESS CHAR(25),
    STREET_NUMBER CHAR(16),
    STREET_NAME CHAR(52),
    STREET_TYPE CHAR(10),
    POST_DIRECTIONAL CHAR(6),
    PHONE CHAR(42),
    PROGRAM_CATEGORY CHAR(10),
    PROGRAM_CATEGORY_DESCRIPTION CHAR(92),
    INSPECTION_DATE DATE,
    INSPECTION_TYPE CHAR(4),
    INSPECTION_DESC CHAR(102),
    SERIAL_NUM CHAR(20),
    ACTION_CODE CHAR(6),
    ACTION_DESC CHAR(102),
    RESULT_CODE CHAR(6),
    RESULT_DESC CHAR(102),
    VIOLATION_CODE CHAR(14),             
    VIOLATION_DESC CHAR(202),
    INSPECTION_MEMO CHAR(502)
)

In [ ]:
%%sql
COPY insp FROM '/home/jovyan/work/abq-food-inspections-fy2010.csv'
CSV
HEADER
QUOTE '"'
DELIMITER ',';

In [ ]:
%%sql
SELECT COUNT(*) FROM insp;

## Problem 3 - explore your data (30 points)

Now that the data is loaded, ask some questions of it!  Identify key variables of interest and note their ranges along with other useful descriptive statistics.  Choose and define a few lines of inquiry, execute queries for each, and describe what you find in your result sets.  Use any query techniques we've seen in class, including aggregate functions, transformations, subqueries, or others as appropriate. 

If you find interesting patterns, adding plots to your exploration would be useful.

Insert a combination of code and text/markdown cells here to explore your data.

### Create index to make query runs faster

In [ ]:
%%sql
DROP INDEX IF EXISTS idx_violat_type;
CREATE INDEX idx_violat_type ON insp (nature_of_business, violation_code, violation_desc);

In [ ]:
%%sql

SELECT nature_of_business, count(*) as count_total
FROM insp
GROUP BY nature_of_business
ORDER BY count(*) DESC
LIMIT 20;

### It seems that Restaurant is the most common type of business; therefore it is very possible that this type of business will be the category that commit most violations. 

### To find the count of each type of violations, I want to exlude records that indicates there were no violations found, and also the ones that are not specified.

In [ ]:
%%sql
SELECT nature_of_business, violation_code, violation_desc, count(*)
FROM insp
WHERE violation_desc != 'No Violations Found' and violation_desc != 'Additional Comments' 
GROUP BY nature_of_business, violation_code, violation_desc
ORDER BY count(*) DESC
LIMIT 20;

In [ ]:
%%sql
SELECT nature_of_business, COUNT(*)as count_violation
FROM insp
WHERE violation_desc != 'No Violations Found' and violation_desc != 'Additional Comments' 
GROUP BY nature_of_business
ORDER BY COUNT(*) DESC
LIMIT 20

### We can see from above table indicating that Restaurant is the type of the business that commit most violations. 

### Now, I want to see that what are the top 10 most commonly commited types of violation for restaurants specifically.

In [ ]:
%matplotlib inline

In [ ]:
%%sql
SELECT nature_of_business, violation_desc, count(*)
FROM insp
WHERE nature_of_business = 'RESTAURANT' and violation_desc != 'No Violations Found' and violation_desc != 'Additional Comments' 
GROUP BY nature_of_business, violation_desc
LIMIT 10;

In [ ]:
result = _
result.bar()

## Bonus (10 points)

We have seen a few simple techniques for normalizing data in class.  Choose a column or set of columns that might be redundant and normalize them out to another table:

* Identify the columns to normalize in the source data table
* Create one or more new tables to hold the normalized records
* Extract data into the new tables
* Re-insert the new foreign key references from the new table into the source table

### Extract information about whether there were one or multiple inspections taken on the same day.

In [ ]:
%%sql
SELECT DISTINCT OWNER_KEY, SERIAL_NUM
FROM insp
ORDER BY OWNER_KEY
LIMIT 10;

In [ ]:
%%sql
SELECT DISTINCT INSPECTION_DATE, SERIAL_NUM
FROM insp
ORDER BY INSPECTION_DATE
LIMIT 10;

In [ ]:
%%sql
DROP TABLE IF EXISTS inspections;
CREATE TABLE inspections (
    inspection_date DATE,
    owner_key CHAR(26),
    serial_num CHAR(20)
);

In [ ]:
%%sql
INSERT INTO inspections (inspection_date, owner_key, serial_num)
SELECT inspection_date, owner_key, serial_num FROM insp;

In [ ]:
%%sql
SELECT * FROM inspections LIMIT 10;